# Cases in Malaysia
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide: false
- comments: false
- permalink:/covid-my-overview/

In [0]:
#hide
#@title Import modules
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML
# import ipywidgets as widgets
# from jinja2 import Template

In [0]:
#hide
#@title Load datasets from source and define functions
#@markdown source: [Novel Coronavirus (COVID-19) Cases, provided by JHU CSSE](https://github.com/CSSEGISandData/COVID-19)
#@markdown * `cases_global` Global dataset by type
#@markdown * `country_cases_df` Filter cases by country and dataset type

# source data
# [Novel Coronavirus (COVID-19) Cases, provided by JHU CSSE](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'
time_series_url = f'{base_url}/csse_covid_19_time_series'
covid_time_series_df = lambda type: pd.read_csv(f'{time_series_url}/time_series_covid19_{type}_global.csv')

# confirmed, deaths, recovered
def cases_global(type):
  _dff = covid_time_series_df(type)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

country_cases_df = lambda _df, country: _df[_df['Country'] == country][['Date', 'Cases']]

In [0]:
#hide
#@title Prepare datasets
# Country filter
global_cases = {
    'confirmed': cases_global('confirmed'),
    'deaths': cases_global('deaths'),
    'recovered': cases_global('recovered')
}

def country_cases(country): 
  confirmed_df = country_cases_df(global_cases['confirmed'], country)
  deaths_df = country_cases_df(global_cases['deaths'], country)
  recovered_df = country_cases_df(global_cases['recovered'], country)

  active_df = pd.concat([confirmed_df['Date'], confirmed_df['Cases'] - deaths_df['Cases'] - recovered_df['Cases']], axis=1)
  
  return {
    'country': country,
    'active': active_df,
    'confirmed': confirmed_df,
    'deaths': deaths_df,
    'recovered': recovered_df
  }

cases_summary = lambda c: { # c country_cases(country)
    'country': c['country'],
    'date_latest': c['confirmed']['Date'].max(),
    's_date_latest': c['confirmed']['Date'].max().strftime("%B %d, %Y"), #.strftime('%m/%d/%Y')
    'total_active': c['active'].iloc[-1]['Cases'],
    'total_confirmed': c['confirmed'].iloc[-1]['Cases'],
    'total_deaths': c['deaths'].iloc[-1]['Cases'],
    'total_recovered': c['recovered'].iloc[-1]['Cases'],
    'new_active': c['active'].diff().iloc[-1]['Cases'],
    'new_confirmed': c['confirmed'].diff().iloc[-1]['Cases'],
    'new_deaths': c['deaths'].diff().iloc[-1]['Cases'],
    'new_recovered': c['recovered'].diff().iloc[-1]['Cases'],
}

In [0]:
#hide
#@title Define plot functions { form-width: "100px" }
def show_summary(country_cases):
  summary = cases_summary(country_cases)
  return HTML(
    f'<div style="height: 30px; width: 600px; margin: 0 auto;"><span style="font-size:0.8em;">Summary of {summary["country"]} COVID19 cases as of {summary["s_date_latest"]} 12PM</span></div>'
    f'<div style="width: 600px; margin: 0 auto;">'
    f'<div style="float: left; width: 150px">Confirmed Cases<br/><h1>{summary["total_confirmed"]:,}</h1>(+{summary["new_confirmed"]:,.0f})</div>'
    f'<div style="float: right; width: 150px">Deaths<br/><h1>{summary["total_deaths"]:,}</h1>{summary["total_deaths"]/summary["total_confirmed"]:.2%} (+{summary["new_deaths"]:,.0f})</div>'
    f'<div style="float: right; width: 150px">Recovered<br/><h1>{summary["total_recovered"]:,}</h1>{summary["total_recovered"]/summary["total_confirmed"]:.2%} (+{summary["new_recovered"]:,.0f})</div>'
    f'<div style="float: right; width: 150px">Active<br/><h1>{summary["total_active"]:,}</h1>{summary["total_active"]/summary["total_confirmed"]:.2%} ({summary["new_active"]:,.0f})</div>'
    f'</div>'
  )

def plot_cases(cases_df):
  _plot = alt.Chart(cases_df).mark_bar().encode(
    x='Date:T',
    y='Cases:Q',
    tooltip=list(cases_df)
  )
  return _plot


## COVID-19: Malaysia at a Glance

### Malaysia Movement Control Order
> The 2020 Malaysia movement control order, commonly referred to as the MCO, is a cordon sanitaire implemented as a preventive measure by the federal government of Malaysia in response to the COVID-19 pandemic in the country on 18 March 2020.

In [0]:
#hide
#@title Define functions for MCO { form-width: "100px" }
#@markdown * my_cases
#@markdown * my_summary

#country = 'Malaysia'
my_cases = country_cases('Malaysia')
my_summary = cases_summary(my_cases)

def plot_mco_date(date, label, max):
  _df = pd.DataFrame({'Date': [date, date], 'Cases': [0, max]})
  _base_ref = alt.Chart(_df).encode(x='Date:T', y='Cases:Q')
  return (_base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3])  + 
          _base_ref.transform_filter(alt.datum['Cases'] > 0).mark_text(text=label, dx=-20, dy=-10, angle=270))

def plot_mco_cases(case, label, max):
  _df = pd.DataFrame({'Cummulative Cases': [case, case], 'Cases': [0.1, max]})
  _base_ref = alt.Chart(_df).encode(x='Cummulative Cases:Q', y='Cases:Q')
  return (_base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3])  + 
          _base_ref.transform_filter(alt.datum['Cases'] > 1).mark_text(text=label, dx=-20, dy=-10, angle=270))

_plot_mco_date = lambda n: plot_mco_date('2020-03-18', 'MCO1', n) + plot_mco_date('2020-04-01', 'MCO2', n) \
                                        + plot_mco_date('2020-04-15', 'MCO3', n) + plot_mco_date('2020-04-29', 'MCO4', n)


_plot_mco_case = plot_mco_cases(790, 'MCO1', 1000) + plot_mco_cases(2908, 'MCO2', 1000) + plot_mco_cases(5072, 'MCO3', 1000) + plot_mco_cases(5945, 'MCO4', 1000)

# Add latest data which not exist in source
_latest_case = {
    'date': np.datetime64('2020-05-04'),
    'confirmed': 6353,
    'deaths': 105,
    'recovered': 4484
}
if len(my_cases['confirmed'][my_cases['confirmed']['Date'] == _latest_case['date']]) == 0:
  my_cases['active'] = my_cases['active'].append(pd.DataFrame([[_latest_case['date'], _latest_case['confirmed'] - _latest_case['deaths'] - _latest_case['recovered']]], columns = ['Date', 'Cases']), ignore_index=True)
  my_cases['confirmed'] = my_cases['confirmed'].append(pd.DataFrame([[_latest_case['date'], _latest_case['confirmed']]], columns = ['Date', 'Cases']), ignore_index=True)
  my_cases['deaths'] = my_cases['deaths'].append(pd.DataFrame([[_latest_case['date'], _latest_case['deaths']]], columns = ['Date', 'Cases']), ignore_index=True)
  my_cases['recovered'] = my_cases['recovered'].append(pd.DataFrame([[_latest_case['date'], _latest_case['recovered']]], columns = ['Date', 'Cases']), ignore_index=True)

In [52]:
#hide_input
show_summary(my_cases)

In [53]:
#hide_input
_plot = plot_cases(my_cases['confirmed'])
_plot_recovered = plot_cases(my_cases['recovered']).mark_line(color='green')
_plot_deaths = plot_cases(my_cases['deaths']).mark_line(color='red')

alt.layer(_plot + _plot_mco_date(6500), _plot_recovered, _plot_deaths).properties(
    width=600,
    title=f'Cummulative COVID-19 cases in Malaysia (n = {my_summary["total_confirmed"]})'
)

alt.LayerChart(...)

### Is the curve is flatterning in Malaysia?
> Inflection-sensitive chart for detecting successful interventions, from the article "How To Tell If We're Beating COVID-19". Please refer _minutephysics_ for [How To Tell If We're Beating COVID-19](https://youtu.be/54XLXg4fYsc)

In [0]:
#hide
#@title Define faltterning plot
def plot_flattern(cases):
  _data = cases['confirmed'].copy()
  _data['New Cases'] = _data['Cases'].diff()
  _data['Average New Cases'] = round(_data['New Cases'].rolling(window=7).mean(),0)
  _data = _data[(_data['Cases'] > 100) & (_data['Average New Cases'] > 0)]
  _plot = alt.Chart(_data).mark_line().encode(
      alt.X('Cases:Q', scale=alt.Scale(type='log'), title = 'Cummulative Confirmed Cases (Log Scale)'), 
      alt.Y('Average New Cases:Q', scale=alt.Scale(type='log', domain=[0.1, 1000]), title = '7 Days Mean New Cases (Log Scale)')
  )
  return _plot.properties(
    title = f'Trajectory of COVID-19 Confirmed Cases in {cases["country"]}',
    width = 600
  )

In [55]:
#hide_input
_plot = plot_flattern(my_cases)
(_plot + _plot_mco_case + plot_flattern(country_cases('Korea, South')).mark_line(color='mediumvioletred')).properties(
    title = f'Trajectory of COVID-19 Confirmed Cases in Malaysia (blue) vs South, Korea (violet)'
)

alt.LayerChart(...)

In [56]:
#hide_input
#@title Plot daily reported cases
rolling_window = 7
_data = my_cases['confirmed'].copy()
_data['New Cases'] = _data['Cases'].diff()
_data['Average New Cases'] = round(_data['New Cases'].rolling(window=rolling_window).mean(),0)
_plot_bar = alt.Chart(_data).mark_bar().encode(
    x='Date:T',
    y='New Cases:Q',
    tooltip = list(_data)
)

_plot_line = alt.Chart(_data).mark_line(color = 'red').encode(
    x='Date:T',
    y='Average New Cases:Q',
    tooltip = list(_data)
)

(_plot_bar + _plot_line + _plot_mco_date(round(_plot_bar.data['New Cases'].max() / 100, 1) * 100)).properties(
    title = f'Daily Reported Cases (with {rolling_window} days average) in Malaysia',
    width = 600
)

alt.LayerChart(...)

In [57]:
#hide_input
#@title Plot remaining active cases
rolling_window = 7
_data = my_cases['active'].copy()
_data['Average Cases'] = round(_data['Cases'].rolling(window=rolling_window).mean(),0)
_plot_bar = alt.Chart(_data).mark_bar().encode(
    x='Date:T',
    y='Cases:Q',
    tooltip = list(_data)
)

_plot_line = alt.Chart(_data).mark_line(color = 'red').encode(
    x='Date:T',
    y='Average Cases:Q',
    tooltip = list(_data)
)

(_plot_bar + _plot_line + _plot_mco_date(round(_plot_bar.data['Cases'].max() / 100, 1) * 100)).properties(
    title = f'Daily Active Cases (with {rolling_window} days average) in Malaysia',
    width = 600
)

alt.LayerChart(...)

In [58]:
#hide
df_states = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTzT9vUJNiKV2yN4sb_VvxKcq-B2triWGPE74rfUT4XOsF-5qsB1tM6OfMPVKiRHX95tE9tPubdTbxY/pub?gid=1726267961&single=true&output=csv", parse_dates=['Date'])
df_states.set_index('Date').sum(axis=1)

Date
3/27    130
3/28    159
3/29    150
3/30    156
3/31    140
4/1     142
4/2     208
4/3     217
4/4     150
4/5     179
4/6     131
4/7     170
4/8     156
4/9     109
4/10    118
4/11    184
4/12    153
4/13    134
4/14    170
4/15     85
4/16    110
4/17     69
4/18     54
4/19     84
4/20     36
4/21     57
4/22     50
dtype: int64

In [59]:
#hide
df_districts = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTzT9vUJNiKV2yN4sb_VvxKcq-B2triWGPE74rfUT4XOsF-5qsB1tM6OfMPVKiRHX95tE9tPubdTbxY/pub?gid=1667946793&single=true&output=csv")
col_latest = df_districts.columns.values[-1]

df_districts[['Districts', 'State', col_latest]].sort_values(col_latest, ascending=False).reset_index(drop=True).head(10)

,Districts,State,4/20
0,Lembah Pantai,KUL,629
1,Hulu Langat,SGR,446
2,Petaling,SGR,374
3,Seremban,NSN,307
4,Kuching,SRW,264
5,Kluang,JHR,222
6,Johor Bahru,JHR,193
7,Klang,SGR,173
8,Kepong,KUL,159
9,Gombak,SGR,144


In [60]:
#hide
df_districts_last2 = df_districts.set_index(['Districts', 'State']).transpose().tail(2)
df_districts_new = df_districts_last2.diff().tail(1).transpose()
df_districts_new = df_districts_new.sort_values(by=df_districts_new.columns[0], ascending=False).head(10)
df_districts_new[df_districts_new[df_districts_new.columns[0]] > 0]

,,4/20
Districts,State,
Putrajaya,PJY,13.0
Kuching,SRW,4.0
Seremban,NSN,3.0
Kuala Langat,SGR,2.0
Bintulu,SRW,2.0
Kepong,KUL,2.0
Kuantan,PHG,2.0
Johor Bahru,JHR,1.0
Melaka Tengah,MLK,1.0
